<a href="https://colab.research.google.com/github/leemordechai/FLAME/blob/master/EI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from google.colab import files
from numpy import nan as Nan
import datetime
from collections import Counter

In [2]:
raw_file = files.upload()
org_raw_df = pd.read_csv('EI-finds.csv')

raw_df = org_raw_df[org_raw_df['Consider'] == 1]

raw_df['ID'] = raw_df['ID'].astype(int)
raw_df['Qualifier'] = raw_df['Qualifier'].fillna('')
raw_df['Notes'] = raw_df['Notes'].fillna('')
raw_df['Comment'] = raw_df['Comment'].fillna('')
raw_df['Type'] = raw_df['Type'].fillna('')
raw_df['Symbol'] = raw_df['Symbol'].fillna('')
raw_df['Inventory'] = raw_df['Inventory'].fillna('')
raw_df['Reference'] = raw_df['Reference'].fillna('')

Saving EI-finds.csv to EI-finds.csv


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

In [3]:
raw_df.head(2)

,ID,Consider,Location,Latitude,Longitude,Qualifier,Hoards/Excavation/Single Finds,Hoard Find Date,Excavation Date Beginning,Excavation Date end,...,End Date,Mint,Comment,Type,Symbol,Inventory,Reference,Quantity,Citation,Notes
0,1,1,Or 'Aqiva,32.507559,34.920609,Agr Complex,Excavation,NaN,NaN,NaN,...,750,al-Ramla,,,,92996,SNAT 65,1.0,'Ad Or 'Aqiva Farm Irrig Complex Atiqot 61 199...,
1,1,1,Or 'Aqiva,32.507559,34.920609,Agr Complex,Excavation,NaN,NaN,NaN,...,750,al-Ramla,,,,92998,SNAT 70,1.0,'Ad Or 'Aqiva Farm Irrig Complex Atiqot 61 199...,


In [0]:
def setting_coin_finds(finds):
  cols_finds = ['hoard_id', 'name', 'startDate', 'endDate', 'type_find', 'hoard?', 'excavation?', 'single?', 'num_coins', 'num_known_coins', 'num_unknown_coins', 'year_found',
    'year_end_found', 'comments', 'lat', 'long', 'certainty', 'references', 'owner', 'created', 'imported']
  coin_finds = pd.DataFrame(columns=cols_finds)
  
  coin_finds['hoard_id'] = ['EI-PP-' + str(i) for i in range(1,147)]
  find_names = []
  start_dates = []
  end_dates = []
  find_types = []
  num_coins = []
  hoard_find_date = []
  excav_date_begin = []
  excav_date_end = []
  lats = []
  longs = []
  references = []
  for i in range(1,147):

    minor_df = raw_df[raw_df['ID'] == i]
    try:
      one_entry = minor_df.iloc[0]
    except: 
      raw_df[raw_df['ID'] == 138].append([Nan]) # just to get an empty row
      
    result = one_entry['Location']
    if one_entry['Qualifier'] != '': result = result + ", " + one_entry['Qualifier']
    if one_entry['Notes'] != '': 
      if i == 18: result = result + ", Isolates" # exception based on catalog
      else: result = result + ", " + one_entry['Notes']
    find_names.append(result)
    
    start_dates.append(minor_df['Start Date'].dropna().min())
    end_dates.append(minor_df['End Date'].dropna().max())
    find_types.append(one_entry['Hoards/Excavation/Single Finds'])
    num_coins.append(minor_df['Quantity'].sum())
    
    hoard_find_date.append(one_entry['Hoard Find Date'])
    excav_date_begin.append(one_entry['Excavation Date Beginning'])
    excav_date_end.append(one_entry['Excavation Date end'])    
    
    lats.append(one_entry['Latitude'])
    longs.append(one_entry['Longitude'])
    
    ref_basic = one_entry['Citation']
    if one_entry['Notes'] != '':
      ref_basic = ref_basic + ' - ' + one_entry['Notes']
    references.append(ref_basic)
    
  # populating data frame
  coin_finds['name'] = find_names
  coin_finds['startDate'] = start_dates
  coin_finds['endDate'] = end_dates
  
  excavations = [1 if i == 'Excavation' else 0 for i in find_types]
  hoards = [1 if i == 'Hoard' else 0 for i in find_types]
  singles = [0] * 146
  singles[17] = 1 # exception based on catalog, only single find
  coin_finds['hoard?'] = hoards
  coin_finds['excavation?'] = excavations
  coin_finds['single?'] = singles
  coin_finds['num_coins'] = num_coins
  coin_finds['lat'] = lats
  coin_finds['long'] = longs
  coin_finds['references'] = references
  coin_finds['created'] = datetime.datetime(2019, 4, 18)
  coin_finds['imported'] = datetime.datetime.now()
  coin_finds['owner'] = 'PeterPhilips'
  coin_finds['certainty'] = 'highest'
  
  
  final_start = []
  final_end = []
  for years in range(len(hoard_find_date)):
    i = hoard_find_date[years]
    j = excav_date_begin[years]
    k = excav_date_end[years]
    
    if (i == '') or (i != i):
      final_start.append(j)
      final_end.append(k)
    else:
      try:
        final_start.append(i[:4])
        final_end.append(i[5:9])
      except:
        final_start.append('')
        final_end.append('')
      
  coin_finds['year_found'] = final_start
  coin_finds['year_end_found'] = final_end  
  
  return coin_finds  

In [0]:
finds = setting_coin_finds(raw_df)
finds.at[83, 'certainty'] = 'lower'

In [0]:
def setting_coin_groups(raw_finds):
  cols = ['hoard_id', 'coin_group_id', 'start_year', 'end_year', 'revised_start', 'revised_end', 'ruler', 'revised_ruler',
    'denomination', 'num_coins', 'mint', 'imported', 'created', 'updated', 'comments']
  coin_groups = pd.DataFrame(columns=cols)
  
  hoard_id = raw_finds['ID'].apply(lambda x: 'EI-PP-' + str(x))
  coin_groups['hoard_id'] = hoard_id
  coin_groups['start_year'] = raw_finds['Date']
  coin_groups['revised_start'] = raw_finds['Start Date']
  coin_groups['revised_end'] = raw_finds['End Date']
  coin_groups['mint'] = raw_finds['Mint']
  coin_groups['denomination'] = raw_finds['Denom.']
  coin_groups['created'] = datetime.datetime(2019, 4, 18)
  coin_groups['imported'] = datetime.datetime.now()
  coin_groups['updated'] = datetime.datetime.now()
  coin_groups['num_coins'] = raw_finds['Quantity']
  coin_groups['comments'] = comment_builder(raw_finds)
  
  c = Counter(set(coin_groups.hoard_id))
  for i in range(len(coin_groups)):
    entry = coin_groups.iloc[i]
    id_num = c[entry['hoard_id']]
    coin_groups.at[entry.name, 'coin_group_id'] = entry['hoard_id'] + '-' + str(id_num)
    c[entry['hoard_id']] += 1
  
  return coin_groups

In [0]:
def comment_builder(raw_finds):
  comments = list(raw_finds['Comment'])
  types = list(raw_finds['Type'])
  symbols = list(raw_finds['Symbol'])
  invent = list(raw_finds['Inventory'])
  refs = list(raw_finds['Reference'])

  entries = []
  for i, j, k, l, m in zip(comments, types, symbols, invent, refs):
    entry = ''
    if i != '': entry = entry + i + '/'
    if j != '': entry = entry + 'Type: ' + j + '/'
    if k != '': entry = entry + 'Symbol: ' + k + '/'
    if l != '': entry = entry + 'Inventory: ' + l + '/'
    if m != '': entry = entry + 'Reference: ' + m + '/'
    entries.append(entry)
  return entries
  
coins = setting_coin_groups(raw_df)
#raw_df['Comment'].map(str) + '/ Type: ' + raw_df['Type'].map(str) + '/ Symbol: ' + raw_df['Symbol'].map(str) + '/ Inventory'

In [0]:
denomination_conversions = {"1/2 follis":"20 nummi", "Fals Umayyad/Abbasid":"fals", "Fals [Abbasid]":"fals", "Fals [Lead]":"fals [lead]", "fals Umayyad/Abbasid":"fals"}
mint_conversions = {'Hims':'Homs', 'Darabjird':'Darabgird (DA)', "al-Madinat Ma'din":"al-Madina ma'din amir al-mu'minin", 
                    'Suq al-Ahwaz':'Sūq al-Ahwaz', 'Marw':'Marw (ML/MLWY/MRW)', 'al-Basra':'Al-Baṣra',
                    'Unknown (Al-Jazira)':'Unknown (al-Jazira)', 'Bayt Jibrin':'Bayt Jabrin', 'Ardashir Khurra':'Ardašir-Xwarrah (ART)',
                    'Nicomedia':'Nikomedia', 'Scythopolis/Baysan':'Baysan', 'Azarbaijan':'Azerbaijān (AT)', 'Iliya':'Iliya (Ayla)',
                    'Neapolis':'Neapolis (Palestine)', 'al-Urdunn [Tabariya]':'Tabariya', 'Unknown (Palestine or Jordan)': 'Unknown (Palestine/Jordan)',
                    # following have been added based on correspondence with Peter Philips on April 20-23, 2019
                    'al-Iskandariyah':'Alexandria ad Aegyptum', 'Cyprus':'Salamis-Konstantia', 'Ifriqiya':'al-Qayrawan',
                    'Kirman':'Kirmān (KRMAN)', 'Mine of the Commander of the Believers in the Hijaz':'Dimashq',
                    'Misr':'al-Fustat', 'Filastin':'al-Ramla', 'Istakhr':'Iṣṭaḵr (ST/STHL)', "al-Rusafa":"al-Madina ma'din amir al-mu'minin",
                    'al-Andalus':'Cordoba', 'Sarafa':'Diocletianopolis', 'Sijistan':'Sīstān (SK/SKSTN)', 'Sarmin (Uncertain)':'Sarmin',
                   'Jabrin':'Jibrin (Uncertain)'}

#based on correspondence with Peter Philips on April 20-23, 2019 I added the following mints:
# 'al-Mawsil', 'Sarmin', 'Balih', 'Diocletianopolis', 'Jibrin (Uncertain)'


coins['denomination'] = coins['denomination'].replace(denomination_conversions)
coins['mint'] = coins['mint'].replace(mint_conversions)

#missing = 'Sarafa', 'al-Andalus', 'Irminiya', 'al-Rusafa', 'Istakhr', 'Filastin','Misr', 'Mine of the Commander of the Believers in the Hijaz','al-Mawsil',
#          'Cyprus', 'Balih' [=Balis?], 'Ifriqiya', 'Kirman', 'Sijistan', 'Sarmin (Uncertain)', 'Jabrin',

#replacements = 'Hims':'Homs', 'Darabjird':'Darabgird (DA)', 'al-Madinat Ma'din':'al-Madina ma'din amir al-mu'minin', 
#                'Suq al-Ahwaz':'Sūq al-Ahwaz', 'Marw':'Marw (ML/MLWY/MRW)', 'al-Basra':'Al-Baṣra'
#                'Unknown (Al-Jazira)':'Unknown (al-Jazira)', 'Bayt Jibrin':'Bayt Jabrin', ''Ardashir Khurra'':'Ardašir-Xwarrah (ART)',
#                'Nicomedia':'Nikomedia', 'Scythopolis/Baysan':'Baysan', 'Azarbaijan':'Azerbaijān (AT)', 'Iliya':'Iliya (Ayla)',
#                'Neapolis':'Neapolis (Palestine)', 'al-Urdunn [Tabariya]':'Tabariya', 'Unknown (Palestine or Jordan)': 'Unknown (Palestine/Jordan)'

#did these:  
#to_code = 'Unknown (Palestine)', 'Unknown (Sasanian Empire)', 'Unknown (Palestine/Jordan)', 'Unknown (al-Jazira)', 
#          'Unknown (Greater Syria)', Unknown (North Syria), 'Unknown (Eastern Umayyad)'


In [0]:
#set(coins.mint) # for testing

In [0]:
#coins.iloc[60:110]
finds.to_csv('PP-EI-coin_finds.csv')
coins.to_csv('PP-EI-coin_groups.csv')
files.download('PP-EI-coin_finds.csv')
files.download('PP-EI-coin_groups.csv')

In [0]:
coins.head(30)

,hoard_id,coin_group_id,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated,comments
0,EI-PP-1,EI-PP-1-1,n.d.,NaN,700,750,NaN,NaN,fals,1.0,al-Ramla,2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Inventory: 92996/Reference: SNAT 65/
1,EI-PP-1,EI-PP-1-2,n.d.,NaN,700,750,NaN,NaN,fals,1.0,al-Ramla,2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Inventory: 92998/Reference: SNAT 70/
2,EI-PP-2,EI-PP-2-1,n.d.,NaN,700,750,NaN,NaN,fals,1.0,Tabariya,2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Inventory: 106183/Reference: SNAT 331-38/
3,EI-PP-2,EI-PP-2-2,n.d.,NaN,700,750,NaN,NaN,fals,1.0,Tabariya,2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Type: Lion/Inventory: 106184/Reference: SNAT 366/
4,EI-PP-3,EI-PP-3-1,n.d.,NaN,700,750,NaN,NaN,fals,1.0,Sarmin,2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Type: Uncertain mint/Inventory: 109/Reference:...
5,EI-PP-3,EI-PP-3-2,n.d.,NaN,700,750,NaN,NaN,fals,1.0,Unknown (Greater Syria),2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Type: Illegible/Inventory: 110/Reference: -/
50,EI-PP-3,EI-PP-3-3,n.d.,NaN,700,750,NaN,NaN,fals,1.0,al-Ramla,2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Inventory: 151/Reference: W.846-8/
52,EI-PP-3,EI-PP-3-4,n.d.,NaN,700,750,NaN,NaN,fals,1.0,Baalbek,2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Inventory: 112/Reference: W.766-73/
53,EI-PP-3,EI-PP-3-5,n.d.,NaN,700,750,NaN,NaN,fals,1.0,Baalbek,2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Inventory: 113/Reference: W.766-73/
54,EI-PP-3,EI-PP-3-6,n.d.,NaN,700,750,NaN,NaN,fals,1.0,Baalbek,2019-04-24 12:33:21.219321,2019-04-18,2019-04-24 12:33:21.220736,Inventory: 114/Reference: W.766-73/
